In [1]:
import os
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Deconvolution2D, Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator



Using TensorFlow backend.


# Load data

In [2]:
# Load training images
pickle_data = pickle.load(open("Raw_images.p", "rb" ))
train_images = pickle_data["Raw"]

# Load image labels
pickle_data = pickle.load(open("GT_images.p", "rb" ))
labels = pickle_data["GroundTruth"]
for i in range(len(labels)):
    labels[i]=np.float32(labels[i])[:,:,:3]
    unichannel= labels[i][:,:,1]
    labels[i] = unichannel
    
    train_images[i]=np.float32(train_images[i])[:,:,:3]
    
print("Training data Loaded")

# Make into arrays as the neural network wants these
train_images = np.float32(train_images)
labels = np.float32(labels).reshape((-1,80,160,1))

Training data Loaded


In [3]:
#print(sum(labels[0]))
print(train_images.shape)
print(labels.shape)

(3626, 80, 160, 3)
(3626, 80, 160, 1)


# Data conditioning

In [4]:
# Normalize labels - training images get normalized to start in the network
labels = labels / 255

# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.15)
print("Training on: ", len(X_train), " samples")
print("Training on: ", len(X_val), " samples")
train_images=[]
labels=[]

Training on:  3082  samples
Training on:  544  samples


# Model parameters

In [14]:
batch_size = 100
epochs = 15
pool_size = (2, 2)
input_shape = X_train.shape[1:]

# Network

In [6]:
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))

# Below layers were re-named for easier reading of model summary; this not necessary
# Conv Layer 1
model.add(Convolution2D(60, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv1'))

# Conv Layer 2
model.add(Convolution2D(50, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv2'))

# Pooling 1
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 3
model.add(Convolution2D(40, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv3'))
model.add(Dropout(0.2))

# Conv Layer 4
model.add(Convolution2D(30, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv4'))
model.add(Dropout(0.2))

# Conv Layer 5
model.add(Convolution2D(20, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv5'))
model.add(Dropout(0.2))

# Pooling 2
model.add(MaxPooling2D(pool_size=pool_size))

# Conv Layer 6
model.add(Convolution2D(10, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv6'))
model.add(Dropout(0.2))

# Conv Layer 7
model.add(Convolution2D(5, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv7'))
model.add(Dropout(0.2))

# Pooling 3
model.add(MaxPooling2D(pool_size=pool_size))

# Upsample 1
model.add(UpSampling2D(size=pool_size))

# Deconv 1
model.add(Deconvolution2D(10, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[8].output_shape, name = 'Deconv1'))
model.add(Dropout(0.2))

# Deconv 2
model.add(Deconvolution2D(20, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[7].output_shape, name = 'Deconv2'))
model.add(Dropout(0.2))

# Upsample 2
model.add(UpSampling2D(size=pool_size))

# Deconv 3
model.add(Deconvolution2D(30, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[5].output_shape, name = 'Deconv3'))
model.add(Dropout(0.2))

# Deconv 4
model.add(Deconvolution2D(40, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[4].output_shape, name = 'Deconv4'))
model.add(Dropout(0.2))

# Deconv 5
model.add(Deconvolution2D(50, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[3].output_shape, name = 'Deconv5'))
model.add(Dropout(0.2))

# Upsample 3
model.add(UpSampling2D(size=pool_size))

# Deconv 6
model.add(Deconvolution2D(60, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[1].output_shape, name = 'Deconv6'))

# Final layer - only including one channel so 1 filter
model.add(Deconvolution2D(1, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', 
                          output_shape = model.layers[0].output_shape, name = 'Final'))

model.summary()

C:\Users\Doctor\.conda\envs\vision-new\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(60, (3, 3), activation="relu", name="Conv1", strides=(1, 1), padding="valid")`
  import sys
C:\Users\Doctor\.conda\envs\vision-new\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (3, 3), activation="relu", name="Conv2", strides=(1, 1), padding="valid")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Doctor\.conda\envs\vision-new\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(40, (3, 3), activation="relu", name="Conv3", strides=(1, 1), padding="valid")`
  app.launch_new_instance()
C:\Users\Doctor\.conda\envs\vision-new\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, (3, 3), activation="relu", name="Conv4", strides=(1, 1), padding=

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 80, 160, 3)        12        
_________________________________________________________________
Conv1 (Conv2D)               (None, 78, 158, 60)       1680      
_________________________________________________________________
Conv2 (Conv2D)               (None, 76, 156, 50)       27050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 78, 50)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 36, 76, 40)        18040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 76, 40)        0         
_________________________________________________________________
Conv4 (Conv2D)               (None, 34, 74, 30)        10830     
__________

# Data feeding

In [15]:
datagen = ImageDataGenerator()
datagen.fit(X_train)

In [16]:
model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), samples_per_epoch = len(X_train),
                    nb_epoch=epochs, verbose=1, validation_data=(X_val, y_val))

# Save model architecture and weights
model_json = model.to_json()
with open("lanes_CNN_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('lanes_CNN_model.h5')


C:\Users\Doctor\.conda\envs\vision-new\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, validation_data=(array([[[..., steps_per_epoch=30, epochs=15)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/15
30/30 [==============================] - 78s - loss: 0.0086 - val_loss: 0.0078
Epoch 2/15
30/30 [==============================] - 74s - loss: 0.0079 - val_loss: 0.0069
Epoch 3/15
30/30 [==============================] - 69s - loss: 0.0075 - val_loss: 0.0067
Epoch 4/15
30/30 [==============================] - 69s - loss: 0.0071 - val_loss: 0.0063
Epoch 5/15
30/30 [==============================] - 68s - loss: 0.0069 - val_loss: 0.0062
Epoch 6/15
30/30 [==============================] - 67s - loss: 0.0069 - val_loss: 0.0059
Epoch 7/15
30/30 [==============================] - 66s - loss: 0.0067 - val_loss: 0.0059
Epoch 8/15
30/30 [==============================] - 66s - loss: 0.0066 - val_loss: 0.0058
Epoch 9/15
30/30 [==============================] - 65s - loss: 0.0063 - val_loss: 0.0059
Epoch 10/15
30/30 [==============================] - 65s - loss: 0.0065 - val_loss: 0.0056
Epoch 11/15
30/30 [==============================] - 65s - loss: 0.0063 - val_loss: 0.0055
Epoch 12